<a href="https://colab.research.google.com/github/ammarisme/covid-19/blob/master/Create_Directory_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create the input directory structure and copy the minimum and mandatory files to start the rest of the processes in the pipeline.

In [0]:
import os
from os import path
from google.colab import drive

In [4]:
#configurations
#drive.flush_and_unmount()
drive.mount('/drive', force_remount=True)
ROOT = "/drive/My Drive/covid"
INPUT_DIR = "/input_mix"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [0]:
DIR = ROOT+INPUT_DIR
if os.path.exists(DIR) == False:
  print('creating directories...')
  os.makedirs(DIR+'/training/cleaned')
  os.makedirs(DIR+'/training/raw')
  os.makedirs(DIR+'/validation/cleaned')
  os.makedirs(DIR+'/validation/raw')
  drive.flush_and_unmount()
  drive.mount('/drive')
  print('done')

In [0]:
from shutil import copyfile
cleaned_file = ROOT+"/input/validation/cleaned/cleaned_worldwide_cases_27_04.csv"
if os.path.exists(cleaned_file):
  copyfile(ROOT+"/input/validation/cleaned/cleaned_worldwide_cases_27_04.csv", DIR+'/validation/cleaned/cleaned_worldwide_cases_27_04.csv')
else:
  print("file doesn\'t exist", cleaned_file)

In [0]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))

In [9]:
paths = DisplayablePath.make_tree(Path(ROOT))
for path in paths:
    print(path.displayable())

covid/
├── _Common.ipynb
├── common (1).py
├── common.py
├── CV19 Cleaning.ipynb
├── CV19 Dataset Explore.ipynb
├── CV19-result-analysis.ipynb
├── CV19_Dataset.ipynb
├── CV19_Synthesis.ipynb
├── CV19Net.ipynb
├── input/
│   ├── backup_2gb/
│   │   ├── Copy of Copy of pre_filter.pt
│   │   ├── Copy of Copy of pre_transform.pt
│   │   └── Copy of Copy of processed.dt
│   ├── categorical_numerical_mapping.dict
│   ├── meta_dictionary.dict
│   ├── training/
│   │   ├── cleaned/
│   │   │   ├── synthetic_run_0_252011_dt_27_04_wmean_2_wstd_2.csv
│   │   │   ├── synthetic_run_10_252011_dt_27_04_wmean_2_wstd_2.csv
│   │   │   ├── synthetic_run_11_252011_dt_27_04_wmean_2_wstd_2.csv
│   │   │   ├── synthetic_run_12_252011_dt_27_04_wmean_2_wstd_2.csv
│   │   │   ├── synthetic_run_13_252011_dt_27_04_wmean_2_wstd_2.csv
│   │   │   ├── synthetic_run_14_252011_dt_27_04_wmean_2_wstd_2.csv
│   │   │   ├── synthetic_run_15_252011_dt_27_04_wmean_2_wstd_2.csv
│   │   │   ├── synthetic_run_16_252011_dt_27_

In [13]:
import dill       
with open('files.pkl', 'wb') as f:
  dill.dump(paths, f)

TypeError: ignored